In [1]:
import re
import pandas as pd

In [2]:
df = pd.read_csv('../../data/03_extracted/article_mentions_extracted.csv')
df = df[~df['art_id'].str.contains('_TRANS', na=False)]

df.head()


,doc_id,art_id,entity_text,entity_label,pattern_group,full_context,words_before_count,words_after_count
0,6D0C4493,6D0C4493_4,artículos 21,ARTICLE_MULTI,ARTICLE_MENTIONS,Sin perjuicio de los principios que prevén los...,8,30
1,6D0C4493,6D0C4493_4,artículo 4 d,ARTICLE_MULTI,ARTICLE_MENTIONS,la Constitución local la actuación del persona...,30,29
2,6D0C4493,6D0C4493_5,artículo 18,ARTICLE_MULTI,ARTICLE_MENTIONS,El tratamiento para la persona privada de su l...,14,30
3,6D0C4493,6D0C4493_7,artículo 22 d,ARTICLE_MULTI,ARTICLE_MENTIONS,"implique el uso de la violencia , discriminaci...",30,30
4,6D0C4493,6D0C4493_30,artículo 19 d,ARTICLE_MULTI,ARTICLE_MENTIONS,el que por determinación del Juez de Control s...,30,30


In [3]:
def normalizar_texto(texto):
    """Normaliza texto: elimina saltos de línea y espacios extra"""
    texto = texto.replace('\n', ' ')
    texto = texto.replace('\r', ' ')
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

def detectar_cantidad(texto):
    """
    Detecta si hay una cantidad específica en el texto.
    Retorna: (tiene_cantidad, cantidad_numerica)
    """
    numeros_texto = {
        'dos': 2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6,
        'siete': 7, 'ocho': 8, 'nueve': 9, 'diez': 10, 'once': 11,
        'doce': 12, 'trece': 13, 'catorce': 14, 'quince': 15,
        'veinte': 20, 'treinta': 30
    }
    
    # Buscar patrón: "los/las [número] artículos"
    match = re.search(r'\b(?:los?|las?)?\s*(dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|veinte|treinta|\d+)\s*art[íi]culos?', 
                      texto, re.IGNORECASE)
    
    if match:
        cantidad_str = match.group(1).lower()
        cantidad = numeros_texto.get(cantidad_str, int(cantidad_str) if cantidad_str.isdigit() else None)
        return True, cantidad
    
    return False, None

def detectar_tipo_referencia(texto):
    """
    Detecta el tipo de referencia: anterior, siguiente, citado, etc.
    Retorna: tipo de referencia
    """
    # Normalizar texto para búsqueda
    texto_norm = texto.lower()
    
    if re.search(r'anterior|precedente|previo', texto_norm):
        return 'anterior'
    elif re.search(r'siguiente|posterior|subsecuente', texto_norm):
        return 'siguiente'
    elif re.search(r'citado|mencionado', texto_norm):
        return 'citado'
    
    return 'desconocido'

def es_plural(texto):
    """
    Determina si la mención es plural.
    Maneja errores gramaticales como "los artículo" (debería ser "los artículos")
    """
    # Buscar "artículos" (con s) o patrones como "los artículo" (error gramatical pero plural)
    if re.search(r'\bartículos\b', texto, re.IGNORECASE):
        return True
    # Detectar "los/las artículo" (error gramatical pero intención plural)
    if re.search(r'\b(?:los|las)\s+art[íi]culo\b', texto, re.IGNORECASE):
        return True
    return False

print("="*70)
print("PROCESAMIENTO DE MENCIONES DE ARTÍCULOS")
print("="*70)


PROCESAMIENTO DE MENCIONES DE ARTÍCULOS


In [4]:
# ==============================================================================
# 1. ARTICLE_SINGLE
# ==============================================================================
print("\n" + "="*70)
print("1. Procesando ARTICLE_SINGLE...")
print("="*70)

df_single = df[df['entity_label'] == 'ARTICLE_SINGLE'].copy()
resultados_single = []

for idx, row in df_single.iterrows():
    texto = normalizar_texto(row['entity_text'])
    
    match = re.search(r'(\d+)\s*(bis|ter|qu[aá]ter|quinto|sexto|s[eé]ptimo|octavo|noveno|d[eé]cimo)?', 
                      texto, re.IGNORECASE)
    
    if match:
        numero = match.group(1)
        sufijo = match.group(2) if match.group(2) else ""
        
        resultados_single.append({
            'doc_id': row['doc_id'],
            'art_id': row['art_id'],
            'entity_text_original': row['entity_text'],
            'entity_text_normalizado': texto,
            'entity_label': 'ARTICLE_SINGLE',
            'numero_articulo': numero,
            'sufijo': sufijo,
            'articulo_completo': f"{numero} {sufijo}".strip(),
            'full_context': row['full_context']
        })

df_single_procesado = pd.DataFrame(resultados_single)
print(f"✓ Procesadas: {len(df_single_procesado):,} menciones")



1. Procesando ARTICLE_SINGLE...
✓ Procesadas: 182 menciones


In [5]:
# ==============================================================================
# 2. ARTICLE_MULTI
# ==============================================================================
print("\n" + "="*70)
print("2. Procesando ARTICLE_MULTI...")
print("="*70)

df_multi = df[df['entity_label'] == 'ARTICLE_MULTI'].copy()
resultados_multi = []

for idx, row in df_multi.iterrows():
    texto = normalizar_texto(row['entity_text'])
    
    matches = re.findall(r'(\d+)\s*(bis|ter|qu[aá]ter|quinto|sexto|s[eé]ptimo|octavo|noveno|d[eé]cimo)?', 
                         texto, re.IGNORECASE)
    
    for numero, sufijo in matches:
        sufijo = sufijo if sufijo else ""
        
        resultados_multi.append({
            'doc_id': row['doc_id'],
            'art_id': row['art_id'],
            'entity_text_original': row['entity_text'],
            'entity_text_normalizado': texto,
            'entity_label': 'ARTICLE_MULTI',
            'numero_articulo': numero,
            'sufijo': sufijo,
            'articulo_completo': f"{numero} {sufijo}".strip(),
            'full_context': row['full_context']
        })

df_multi_procesado = pd.DataFrame(resultados_multi)
print(f"✓ Menciones originales: {len(df_multi):,}")
print(f"✓ Filas expandidas: {len(df_multi_procesado):,}")



2. Procesando ARTICLE_MULTI...
✓ Menciones originales: 4,857
✓ Filas expandidas: 5,267


In [6]:

# ==============================================================================
# 3. ARTICLE_RANGE
# ==============================================================================
print("\n" + "="*70)
print("3. Procesando ARTICLE_RANGE...")
print("="*70)

df_range = df[df['entity_label'] == 'ARTICLE_RANGE'].copy()
resultados_range = []

for idx, row in df_range.iterrows():
    texto = normalizar_texto(row['entity_text'])
    
    match = re.search(r'(\d+)\s*(?:bis|ter|qu[aá]ter)?\s*al\s*(\d+)', texto, re.IGNORECASE)
    
    if match:
        inicio = int(match.group(1))
        fin = int(match.group(2))
        
        for numero in range(inicio, fin + 1):
            resultados_range.append({
                'doc_id': row['doc_id'],
                'art_id': row['art_id'],
                'entity_text_original': row['entity_text'],
                'entity_text_normalizado': texto,
                'entity_label': 'ARTICLE_RANGE',
                'numero_articulo': str(numero),
                'sufijo': "",
                'articulo_completo': str(numero),
                'rango_inicio': inicio,
                'rango_fin': fin,
                'full_context': row['full_context']
            })

df_range_procesado = pd.DataFrame(resultados_range)
print(f"✓ Menciones originales: {len(df_range):,}")
print(f"✓ Filas expandidas: {len(df_range_procesado):,}")



3. Procesando ARTICLE_RANGE...
✓ Menciones originales: 16
✓ Filas expandidas: 86


In [7]:
# ==============================================================================
# 4. ARTICLE_RELATIVE - PROCESAMIENTO COMPLETO
# ==============================================================================
print("\n" + "="*70)
print("4. Procesando ARTICLE_RELATIVE...")
print("="*70)

df_relative = df[df['entity_label'] == 'ARTICLE_RELATIVE'].copy()

resultados_relative_calculado = []  # Con número calculado
resultados_relative_sin_numero = []  # Sin número (plurales sin cantidad o ambiguos)

# Contadores para estadísticas
contador_singular = 0
contador_plural_con_cantidad = 0
contador_plural_sin_cantidad = 0
contador_ambiguo = 0

for idx, row in df_relative.iterrows():
    texto_original = row['entity_text']
    texto = normalizar_texto(texto_original)
    art_id = row['art_id']
    
    # Extraer el número del artículo actual desde art_id
    match_art_id = re.search(r'_(\d+)$', art_id)
    
    if not match_art_id:
        continue
    
    numero_actual = int(match_art_id.group(1))
    
    # Detectar tipo de referencia (anterior, siguiente, citado)
    tipo_ref = detectar_tipo_referencia(texto)
    
    # Detectar si hay cantidad específica
    tiene_cantidad, cantidad = detectar_cantidad(texto)
    
    # ═════════════════════════════════════════════════════════════════════════
    # CASO 1: CON CANTIDAD ESPECÍFICA (ej: "los dos artículos anteriores")
    # ═════════════════════════════════════════════════════════════════════════
    if tiene_cantidad and cantidad:
        contador_plural_con_cantidad += 1
        
        if tipo_ref == 'anterior':
            # Generar múltiples filas hacia atrás
            for i in range(1, cantidad + 1):
                numero_referido = numero_actual - i
                if numero_referido > 0:
                    resultados_relative_calculado.append({
                        'doc_id': row['doc_id'],
                        'art_id': row['art_id'],
                        'entity_text_original': texto_original,
                        'entity_text_normalizado': texto,
                        'entity_label': 'ARTICLE_RELATIVE',
                        'numero_articulo_actual': numero_actual,
                        'tipo_referencia': f"anteriores_cantidad_{cantidad}",
                        'numero_articulo': str(numero_referido),
                        'sufijo': "",
                        'articulo_completo': str(numero_referido),
                        'puede_calcular': True,
                        'razon': f'Calculado - {cantidad} anteriores',
                        'full_context': row['full_context']
                    })
        
        elif tipo_ref == 'siguiente':
            # Generar múltiples filas hacia adelante
            for i in range(1, cantidad + 1):
                numero_referido = numero_actual + i
                resultados_relative_calculado.append({
                    'doc_id': row['doc_id'],
                    'art_id': row['art_id'],
                    'entity_text_original': texto_original,
                    'entity_text_normalizado': texto,
                    'entity_label': 'ARTICLE_RELATIVE',
                    'numero_articulo_actual': numero_actual,
                    'tipo_referencia': f"siguientes_cantidad_{cantidad}",
                    'numero_articulo': str(numero_referido),
                    'sufijo': "",
                    'articulo_completo': str(numero_referido),
                    'puede_calcular': True,
                    'razon': f'Calculado - {cantidad} siguientes',
                    'full_context': row['full_context']
                })
        
        else:
            # Cantidad especificada pero tipo ambiguo (citados, mencionados)
            resultados_relative_sin_numero.append({
                'doc_id': row['doc_id'],
                'art_id': row['art_id'],
                'entity_text_original': texto_original,
                'entity_text_normalizado': texto,
                'entity_label': 'ARTICLE_RELATIVE',
                'numero_articulo_actual': numero_actual,
                'tipo_referencia': f"{tipo_ref}_cantidad_{cantidad}",
                'numero_articulo': None,
                'sufijo': "",
                'articulo_completo': None,
                'puede_calcular': False,
                'razon': f'Ambiguo - {cantidad} artículos {tipo_ref}',
                'full_context': row['full_context']
            })
    
    # ═════════════════════════════════════════════════════════════════════════
    # CASO 2: PLURAL SIN CANTIDAD (ej: "artículos anteriores")
    # ═════════════════════════════════════════════════════════════════════════
    elif es_plural(texto):
        contador_plural_sin_cantidad += 1
        
        tipo_label = f"{tipo_ref}es_plural" if tipo_ref != 'desconocido' else "plural_sin_tipo"
        
        resultados_relative_sin_numero.append({
            'doc_id': row['doc_id'],
            'art_id': row['art_id'],
            'entity_text_original': texto_original,
            'entity_text_normalizado': texto,
            'entity_label': 'ARTICLE_RELATIVE',
            'numero_articulo_actual': numero_actual,
            'tipo_referencia': tipo_label,
            'numero_articulo': None,
            'sufijo': "",
            'articulo_completo': None,
            'puede_calcular': False,
            'razon': 'Plural - cantidad indeterminada',
            'full_context': row['full_context']
        })
    
    # ═════════════════════════════════════════════════════════════════════════
    # CASO 3: SINGULAR (ej: "artículo anterior", "el artículo siguiente")
    # ═════════════════════════════════════════════════════════════════════════
    else:
        if tipo_ref == 'anterior':
            contador_singular += 1
            numero_referido = numero_actual - 1
            if numero_referido > 0:
                resultados_relative_calculado.append({
                    'doc_id': row['doc_id'],
                    'art_id': row['art_id'],
                    'entity_text_original': texto_original,
                    'entity_text_normalizado': texto,
                    'entity_label': 'ARTICLE_RELATIVE',
                    'numero_articulo_actual': numero_actual,
                    'tipo_referencia': "anterior_singular",
                    'numero_articulo': str(numero_referido),
                    'sufijo': "",
                    'articulo_completo': str(numero_referido),
                    'puede_calcular': True,
                    'razon': 'Calculado - singular',
                    'full_context': row['full_context']
                })
        
        elif tipo_ref == 'siguiente':
            contador_singular += 1
            numero_referido = numero_actual + 1
            resultados_relative_calculado.append({
                'doc_id': row['doc_id'],
                'art_id': row['art_id'],
                'entity_text_original': texto_original,
                'entity_text_normalizado': texto,
                'entity_label': 'ARTICLE_RELATIVE',
                'numero_articulo_actual': numero_actual,
                'tipo_referencia': "siguiente_singular",
                'numero_articulo': str(numero_referido),
                'sufijo': "",
                'articulo_completo': str(numero_referido),
                'puede_calcular': True,
                'razon': 'Calculado - singular',
                'full_context': row['full_context']
            })
        
        else:
            # Singular pero ambiguo (citado, mencionado)
            contador_ambiguo += 1
            resultados_relative_sin_numero.append({
                'doc_id': row['doc_id'],
                'art_id': row['art_id'],
                'entity_text_original': texto_original,
                'entity_text_normalizado': texto,
                'entity_label': 'ARTICLE_RELATIVE',
                'numero_articulo_actual': numero_actual,
                'tipo_referencia': f"{tipo_ref}_singular",
                'numero_articulo': None,
                'sufijo': "",
                'articulo_completo': None,
                'puede_calcular': False,
                'razon': f'Ambiguo - {tipo_ref}',
                'full_context': row['full_context']
            })

# Crear dataframes
df_relative_calculado = pd.DataFrame(resultados_relative_calculado)
df_relative_sin_numero = pd.DataFrame(resultados_relative_sin_numero)



4. Procesando ARTICLE_RELATIVE...


In [8]:
# Ver las tablas 
df_single_procesado.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo,sufijo,articulo_completo,full_context
0,6D0C4493,6D0C4493_159,artículos 42,artículos 42,ARTICLE_SINGLE,42,,42,y no devueltos en el caso previsto en la fracc...
1,3F66D749,3F66D749_101QUATER,artículos 76,artículos 76,ARTICLE_SINGLE,76,,76,respectivo ; El cumplimiento de los requisitos...
2,D61F4C69,D61F4C69_188,artículos 185,artículos 185,ARTICLE_SINGLE,185,,185,obtener el registro para elaborar Programas In...
3,D61F4C69,D61F4C69_190,artículos 185,artículos 185,ARTICLE_SINGLE,185,,185,Las personas físicas que pretendan obtener el ...
4,69638137,69638137_12,artículos 36,artículos 36,ARTICLE_SINGLE,36,,36,condiciones de formular y sugerir las observac...


In [9]:
df_multi_procesado.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo,sufijo,articulo_completo,full_context
0,6D0C4493,6D0C4493_4,artículos 21,artículos 21,ARTICLE_MULTI,21,,21,Sin perjuicio de los principios que prevén los...
1,6D0C4493,6D0C4493_4,artículo 4 d,artículo 4 d,ARTICLE_MULTI,4,,4,la Constitución local la actuación del persona...
2,6D0C4493,6D0C4493_5,artículo 18,artículo 18,ARTICLE_MULTI,18,,18,El tratamiento para la persona privada de su l...
3,6D0C4493,6D0C4493_7,artículo 22 d,artículo 22 d,ARTICLE_MULTI,22,,22,"implique el uso de la violencia , discriminaci..."
4,6D0C4493,6D0C4493_30,artículo 19 d,artículo 19 d,ARTICLE_MULTI,19,,19,el que por determinación del Juez de Control s...


In [10]:
df_range_procesado.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo,sufijo,articulo_completo,rango_inicio,rango_fin,full_context
0,293D2FA5,293D2FA5_15,artículos 15 al 17 d,artículos 15 al 17 d,ARTICLE_RANGE,15,,15,15,17,Para definir las líneas de coordinación respec...
1,293D2FA5,293D2FA5_15,artículos 15 al 17 d,artículos 15 al 17 d,ARTICLE_RANGE,16,,16,15,17,Para definir las líneas de coordinación respec...
2,293D2FA5,293D2FA5_15,artículos 15 al 17 d,artículos 15 al 17 d,ARTICLE_RANGE,17,,17,15,17,Para definir las líneas de coordinación respec...
3,C57B66D6,C57B66D6_9,Artículos 17 al 20 d,Artículos 17 al 20 d,ARTICLE_RANGE,17,,17,17,20,refiere el Artículo 7 de esta Ley . Se ofrecer...
4,C57B66D6,C57B66D6_9,Artículos 17 al 20 d,Artículos 17 al 20 d,ARTICLE_RANGE,18,,18,17,20,refiere el Artículo 7 de esta Ley . Se ofrecer...


In [11]:
df_relative_calculado.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo_actual,tipo_referencia,numero_articulo,sufijo,articulo_completo,puede_calcular,razon,full_context
0,6D0C4493,6D0C4493_113,artículo anterior,artículo anterior,ARTICLE_RELATIVE,113,anterior_singular,112,,112,True,Calculado - singular,**La revisión a que se refiere el artículo ant...
1,6D0C4493,6D0C4493_159,artículo anterior,artículo anterior,ARTICLE_RELATIVE,159,anterior_singular,158,,158,True,Calculado - singular,**Los correctivos disciplinarios aplicables a ...
2,6D0C4493,6D0C4493_160,artículo anterior,artículo anterior,ARTICLE_RELATIVE,160,anterior_singular,159,,159,True,Calculado - singular,**Las correcciones disciplinarias a que se ref...
3,2833199A,2833199A_21,artículo anterior,artículo anterior,ARTICLE_RELATIVE,21,anterior_singular,20,,20,True,Calculado - singular,**Si en el transcurso de la sesión se ausenta ...
4,2833199A,2833199A_57,artículo anterior,artículo anterior,ARTICLE_RELATIVE,57,anterior_singular,56,,56,True,Calculado - singular,**De no reunirse el quórum establecido en el a...


In [12]:
df_relative_sin_numero.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo_actual,tipo_referencia,numero_articulo,sufijo,articulo_completo,puede_calcular,razon,full_context
0,3F66D749,3F66D749_19,los artículos anteriores,los artículos anteriores,ARTICLE_RELATIVE,19,anteriores_plural,None,,None,False,Plural - cantidad indeterminada,Las infracciones señaladas en **los artículos ...
1,3F66D749,3F66D749_26,los artículos precedentes,los artículos precedentes,ARTICLE_RELATIVE,26,anteriores_plural,None,,None,False,Plural - cantidad indeterminada,"candidatos , pero forman parte de los órganos ..."
2,3F66D749,3F66D749_95,los artículos anteriores,los artículos anteriores,ARTICLE_RELATIVE,95,anteriores_plural,None,,None,False,Plural - cantidad indeterminada,"obligadas a realizar , dentro del ámbito de su..."
3,C371528F,C371528F_36,los artículos anteriores,los artículos anteriores,ARTICLE_RELATIVE,36,anteriores_plural,None,,None,False,Plural - cantidad indeterminada,La reconvención sólo será procedente en la con...
4,C57B66D6,C57B66D6_187,los artículos anteriores,los artículos anteriores,ARTICLE_RELATIVE,187,anteriores_plural,None,,None,False,Plural - cantidad indeterminada,La Notaria o Notario está obligado a dar a con...


In [13]:
# Unir todos los dataframes
df_all = pd.concat([df_single_procesado, df_multi_procesado, df_range_procesado, df_relative_calculado])
df_all.head()
print(df_all.shape)



(6435, 15)


In [14]:
# Eliminar duplicados
df_all = df_all.drop_duplicates(subset=['art_id', 'numero_articulo'])
print(df_all.shape)
df_all.head()



(5816, 15)


,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo,sufijo,articulo_completo,full_context,rango_inicio,rango_fin,numero_articulo_actual,tipo_referencia,puede_calcular,razon
0,6D0C4493,6D0C4493_159,artículos 42,artículos 42,ARTICLE_SINGLE,42,,42,y no devueltos en el caso previsto en la fracc...,NaN,NaN,NaN,NaN,NaN,NaN
1,3F66D749,3F66D749_101QUATER,artículos 76,artículos 76,ARTICLE_SINGLE,76,,76,respectivo ; El cumplimiento de los requisitos...,NaN,NaN,NaN,NaN,NaN,NaN
2,D61F4C69,D61F4C69_188,artículos 185,artículos 185,ARTICLE_SINGLE,185,,185,obtener el registro para elaborar Programas In...,NaN,NaN,NaN,NaN,NaN,NaN
3,D61F4C69,D61F4C69_190,artículos 185,artículos 185,ARTICLE_SINGLE,185,,185,Las personas físicas que pretendan obtener el ...,NaN,NaN,NaN,NaN,NaN,NaN
4,69638137,69638137_12,artículos 36,artículos 36,ARTICLE_SINGLE,36,,36,condiciones de formular y sugerir las observac...,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# Guardar los si tienen numero
#df_all.to_csv('../../data/04_matched/article_mentions_with_numbers.csv', index=False, encoding='utf-8-sig')


# Guardar los que no tienen numero 
#df_relative_sin_numero.to_csv('../../data/04_matched/article_mentions_sin_numero.csv', index=False, encoding='utf-8-sig')

In [16]:
# Para los que sí tienen numero matchear obtener la ley que menciona

df_all['articulo_creado'] = 'articulo' + ' '+ df_all['numero_articulo'] + df_all['sufijo'].astype(str)

df_all = df_all[['doc_id','art_id','entity_text_original','entity_text_normalizado','entity_label','numero_articulo','articulo_completo','articulo_creado','full_context']]
df_all.head()

,doc_id,art_id,entity_text_original,entity_text_normalizado,entity_label,numero_articulo,articulo_completo,articulo_creado,full_context
0,6D0C4493,6D0C4493_159,artículos 42,artículos 42,ARTICLE_SINGLE,42,42,articulo 42,y no devueltos en el caso previsto en la fracc...
1,3F66D749,3F66D749_101QUATER,artículos 76,artículos 76,ARTICLE_SINGLE,76,76,articulo 76,respectivo ; El cumplimiento de los requisitos...
2,D61F4C69,D61F4C69_188,artículos 185,artículos 185,ARTICLE_SINGLE,185,185,articulo 185,obtener el registro para elaborar Programas In...
3,D61F4C69,D61F4C69_190,artículos 185,artículos 185,ARTICLE_SINGLE,185,185,articulo 185,Las personas físicas que pretendan obtener el ...
4,69638137,69638137_12,artículos 36,artículos 36,ARTICLE_SINGLE,36,36,articulo 36,condiciones de formular y sugerir las observac...


# Obtener la ley a la que pertenece cada artículo

In [17]:
# ==============================================================================
# PASO 2A: PROMPT MEJORADO PARA LLM
# ==============================================================================

def create_prompt(source_law, source_article, mention_text, numero_calculado, context):
    """
    Crea el prompt mejorado para identificar la ley de una mención de artículo.
    
    Args:
        source_law: Nombre de la ley del documento fuente
        source_article: Artículo fuente donde se encontró la mención
        mention_text: Texto original de la mención (ej: "artículo anterior")
        numero_calculado: Número de artículo calculado (ej: "112") o None
        context: Contexto donde aparece la mención
    """
    
    # Preparar display de la mención
    if numero_calculado and str(numero_calculado).strip():
        mention_display = f"Artículo {numero_calculado}"
        extra_note = f" [Nota: El texto original era '{mention_text}']"
    else:
        mention_display = mention_text
        extra_note = ""
    
    return f"""Eres un experto en análisis de documentos legales mexicanos. Tu tarea es identificar a qué ley pertenece un artículo mencionado.

DOCUMENTO FUENTE:
- Ley: {source_law}
- Artículo donde aparece: {source_article}

ARTÍCULO MENCIONADO:
- {mention_display}{extra_note}

CONTEXTO:
{context}

INSTRUCCIONES:
1. Lee cuidadosamente el contexto para identificar indicadores de la ley
2. Busca frases como:
   • "de la presente Ley" / "de esta Ley" / "del presente Reglamento" → AUTO-REFERENCIA
   • "de la Constitución Política de la Ciudad de México" → LEY ESPECÍFICA
   • "de la Ley de [nombre]" → LEY ESPECÍFICA
   • Sin indicador → AUTO-REFERENCIA (asume misma ley)

3. Si identificas una ley diferente, usa su nombre COMPLETO y EXACTO tal como aparece

EJEMPLOS:
• "conforme al artículo 25 de la presente Ley" → AUTO-REFERENCIA a {source_law}
• "según el artículo 18 de la Constitución Federal" → "Constitución Política de los Estados Unidos Mexicanos"
• "artículo anterior" sin otro contexto → AUTO-REFERENCIA a {source_law}

IMPORTANTE:
- Si NO hay indicación explícita de otra ley → es AUTO-REFERENCIA
- Mantén nombres oficiales completos (no abrevies)
- Tu nivel de confianza depende de qué tan explícito es el contexto

Responde ÚNICAMENTE con JSON (sin markdown, sin ```):
{{
  "ley_identificada": "nombre completo oficial de la ley",
  "es_autorreferencia": true o false,
  "confianza": "alta/media/baja",
  "razon": "frase específica del contexto que te llevó a esta conclusión"
}}"""

# ==============================================================================
# PASO 2B: FUNCIÓN PARA LLAMAR AL LLM
# ==============================================================================

def identify_single_mention(client, source_law, source_article, mention_text, 
                           numero_calculado, context, model="gpt-4o-mini"):
    """
    Identifica la ley para una mención usando LLM.
    
    Args:
        client: Cliente de OpenAI
        source_law: Ley del documento fuente
        source_article: Artículo donde aparece la mención
        mention_text: Texto original de la mención
        numero_calculado: Número de artículo calculado (o None)
        context: Contexto de la mención
        model: Modelo a usar
        
    Returns:
        dict con: ley_identificada, es_autorreferencia, confianza, razon, tokens_used, success
    """
    
    prompt = create_prompt(source_law, source_article, mention_text, numero_calculado, context)
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            response_format={"type": "json_object"}  # Forzar respuesta JSON
        )
        
        content = response.choices[0].message.content.strip()
        
        # Limpiar markdown si existe (por si acaso)
        if content.startswith("```json"):
            content = content[7:]
        if content.startswith("```"):
            content = content[3:]
        if content.endswith("```"):
            content = content[:-3]
        content = content.strip()
        
        result = json.loads(content)
        result['tokens_used'] = response.usage.total_tokens
        result['success'] = True
        
        return result
        
    except json.JSONDecodeError as e:
        return {
            'ley_identificada': '',
            'es_autorreferencia': None,
            'confianza': '',
            'razon': f"ERROR JSON: {str(e)} | Content: {content[:100]}",
            'tokens_used': 0,
            'success': False
        }
    except Exception as e:
        return {
            'ley_identificada': '',
            'es_autorreferencia': None,
            'confianza': '',
            'razon': f"ERROR: {str(e)}",
            'tokens_used': 0,
            'success': False
        }

# ==============================================================================
# PASO 2C: PROCESAR TODAS LAS MENCIONES CON LLM
# ==============================================================================

import time
from tqdm import tqdm

def process_mentions_with_llm(df_processed, articles_df, client, model="gpt-4o-mini", sample_size=None):
    """
    Procesa menciones ya procesadas (con números calculados) usando LLM.
    
    Args:
        df_processed: DataFrame con menciones procesadas (df_all del Paso 1)
        articles_df: DataFrame con información de artículos y documentos
        client: Cliente de OpenAI
        model: Modelo a usar
        sample_size: Número de menciones a procesar (None = todas)
        
    Returns:
        DataFrame con resultados del LLM
    """
    
    # Crear lookup de artículos (art_id → info del documento)
    articles_lookup = articles_df.copy()
    articles_lookup['numero_art'] = articles_lookup['art_id'].str.split('_').str[-1]
    # Deduplicar art_id (quedarnos con la primera ocurrencia)
    articles_lookup = articles_lookup.drop_duplicates(subset='art_id', keep='first')
    articles_lookup = articles_lookup.set_index('art_id')[['nombre', 'numero_art']].to_dict('index')

    # Muestra (si se especifica)
    df_to_process = df_processed.head(sample_size) if sample_size else df_processed
    
    print(f"{'='*70}")
    print(f"PASO 2: IDENTIFICACIÓN DE LEYES CON LLM")
    print(f"{'='*70}")
    print(f"Menciones a procesar: {len(df_to_process):,}")
    print(f"Modelo: {model}")
    print(f"{'='*70}\n")
    
    results = []
    total_tokens = 0
    errores = 0
    start_time = time.time()
    
    for idx, row in tqdm(df_to_process.iterrows(), total=len(df_to_process), desc="Procesando"):
        art_id = row['art_id']
        
        # Buscar información del artículo fuente
        if art_id not in articles_lookup:
            # Si no está en lookup, intentar buscar por doc_id
            doc_matches = articles_df[articles_df['doc_id'] == row['doc_id']]
            if len(doc_matches) == 0:
                continue
            article_info = {
                'nombre': doc_matches.iloc[0]['nombre'],
                'numero_art': doc_matches.iloc[0]['art_id'].split('_')[-1]
            }
        else:
            article_info = articles_lookup[art_id]
        
        # Llamar al LLM
        result = identify_single_mention(
            client=client,
            source_law=article_info['nombre'],
            source_article=f"Artículo {article_info['numero_art']}",
            mention_text=row['entity_text_original'],
            numero_calculado=row.get('numero_articulo'),
            context=row['full_context'],
            model=model
        )
        
        # Agregar información del procesamiento previo
        result['art_id'] = art_id
        result['doc_id'] = row['doc_id']
        result['source_law'] = article_info['nombre']
        result['source_article'] = f"Artículo {article_info['numero_art']}"
        result['entity_text_original'] = row['entity_text_original']
        result['numero_articulo'] = row.get('numero_articulo')
        result['articulo_completo'] = row.get('articulo_completo')
        result['entity_label'] = row.get('entity_label')
        
        results.append(result)
        total_tokens += result.get('tokens_used', 0)
        
        if not result['success']:
            errores += 1
        
        # Pausa cada 10 requests para evitar rate limits
        if len(results) % 10 == 0:
            time.sleep(1.2)
    
    elapsed = time.time() - start_time
    results_df = pd.DataFrame(results)
    
    # Reporte
    print(f"\n{'='*70}")
    print(f"RESULTADOS DEL PROCESAMIENTO LLM")
    print(f"{'='*70}")
    print(f"Tiempo total: {elapsed/60:.2f} minutos")
    print(f"Menciones procesadas: {len(results_df):,}")
    print(f"  ✓ Exitosas: {results_df['success'].sum():,}")
    print(f"  ✗ Con errores: {errores:,}")
    print(f"\nTokens usados: {total_tokens:,}")
    
    # Calcular costo (gpt-4o-mini: $0.15 por 1M tokens input, $0.60 por 1M tokens output)
    # Aproximación: ~70% input, ~30% output
    costo_estimado = (total_tokens * 0.7 / 1_000_000 * 0.15) + (total_tokens * 0.3 / 1_000_000 * 0.60)
    print(f"Costo estimado: ${costo_estimado:.2f} USD")
    
    # Distribución de confianza
    if len(results_df[results_df['success']]) > 0:
        print(f"\nDistribución de confianza:")
        print(results_df[results_df['success']]['confianza'].value_counts().to_string())
    
    print(f"{'='*70}\n")
    
    return results_df


In [18]:
import os
from openai import OpenAI
import json

# Configurar cliente OpenAI
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Configuración
SAMPLE_SIZE = None  # None = procesar todo, o número específico para pruebas
MODEL = "gpt-4o-mini"

print("="*70)
print("PIPELINE COMPLETO: MENCIONES DE ARTÍCULOS")
print("="*70)
print(f"\nConfiguración:")
print(f"  • Modelo: {MODEL}")
print(f"  • Sample: {'Todo' if SAMPLE_SIZE is None else f'{SAMPLE_SIZE:,} menciones'}")
print("="*70)


PIPELINE COMPLETO: MENCIONES DE ARTÍCULOS

Configuración:
  • Modelo: gpt-4o-mini
  • Sample: Todo


In [19]:
# ==============================================================================
# PASO 2: EJECUTAR LLM
# ==============================================================================

# Cargar el catálogo de artículos (necesitamos document_name para cada art_id)
# AJUSTA ESTA RUTA según tu estructura de datos
articles_df = pd.read_csv('../../data/02_catalogs/art_hash.csv')  # O tu archivo equivalente
articles_df.drop_duplicates(subset=['art_id'], inplace=True)

print(f"\nDataset de artículos cargado: {len(articles_df):,} artículos")
print(f"Menciones procesadas (df_all): {len(df_all):,}")


print(articles_df.head())
print(df_all.head())


Dataset de artículos cargado: 30,299 artículos
Menciones procesadas (df_all): 5,816
       art_id    doc_id                                               text  \
0  6D0C4493_1  6D0C4493  Las disposiciones contenidas en este Reglament...   
1  6D0C4493_2  6D0C4493  La aplicación del presente Reglamento correspo...   
2  6D0C4493_3  6D0C4493  Para los efectos del presente Reglamento, adem...   
3  6D0C4493_4  6D0C4493  Sin perjuicio de los principios que prevén los...   
4  6D0C4493_5  6D0C4493  El tratamiento para la persona privada de su l...   

                                              nombre menciona_norm  \
0  Reglamento de la Ley de Centros Penitenciarios...           NaN   
1  Reglamento de la Ley de Centros Penitenciarios...           NaN   
2  Reglamento de la Ley de Centros Penitenciarios...           NaN   
3  Reglamento de la Ley de Centros Penitenciarios...           NaN   
4  Reglamento de la Ley de Centros Penitenciarios...           NaN   

  menciona_ley_materia  


In [20]:
# Ejecutar LLM
df_llm_results = process_mentions_with_llm(
    df_processed=df_all,
    articles_df=articles_df,
    client=client,
    model=MODEL,
    sample_size=SAMPLE_SIZE
)

# Resultados
df_llm_results.head()

PASO 2: IDENTIFICACIÓN DE LEYES CON LLM
Menciones a procesar: 5,816
Modelo: gpt-4o-mini



Procesando: 100%|██████████| 5816/5816 [3:51:25<00:00,  2.39s/it]     


RESULTADOS DEL PROCESAMIENTO LLM
Tiempo total: 231.43 minutos
Menciones procesadas: 5,816
  ✓ Exitosas: 5,816
  ✗ Con errores: 0

Tokens usados: 3,324,939
Costo estimado: $0.95 USD

Distribución de confianza:
confianza
alta    5816



,ley_identificada,es_autorreferencia,confianza,razon,tokens_used,success,art_id,doc_id,source_law,source_article,entity_text_original,numero_articulo,articulo_completo,entity_label
0,Reglamento de la Ley de Centros Penitenciarios...,True,alta,No hay indicación explícita de otra ley en el ...,579,True,6D0C4493_159,6D0C4493,Reglamento de la Ley de Centros Penitenciarios...,Artículo 159,artículos 42,42,42,ARTICLE_SINGLE
1,Ley Procesal Electoral de la Ciudad de Mexico,True,alta,"El contexto menciona 'de la presente Ley', lo ...",561,True,3F66D749_101QUATER,3F66D749,Ley Procesal Electoral de la Ciudad de Mexico,Artículo 101QUATER,artículos 76,76,76,ARTICLE_SINGLE
2,Reglamento de la Ley de Gestion Integral de Ri...,True,alta,"El contexto menciona 'los artículos 185, 186 y...",622,True,D61F4C69_188,D61F4C69,Reglamento de la Ley de Gestion Integral de Ri...,Artículo 188,artículos 185,185,185,ARTICLE_SINGLE
3,Reglamento de la Ley de Gestion Integral de Ri...,True,alta,"El contexto menciona 'los artículos 185, 186 y...",604,True,D61F4C69_190,D61F4C69,Reglamento de la Ley de Gestion Integral de Ri...,Artículo 190,artículos 185,185,185,ARTICLE_SINGLE
4,Reglamento de la Ley de Proteccion y Fomento a...,True,alta,No hay indicación explícita de otra ley en el ...,588,True,69638137_12,69638137,Reglamento de la Ley de Proteccion y Fomento a...,Artículo 12,artículos 36,36,36,ARTICLE_SINGLE


PASO 3. MATCHEAR CON EL CATALOGO DE ARTICULOS

In [21]:
# Mostrar primeras filas
df_llm_results.head()

df_llm_results = df_llm_results[['ley_identificada', 'es_autorreferencia', 'art_id','doc_id','articulo_completo']]
df_llm_results

,ley_identificada,es_autorreferencia,art_id,doc_id,articulo_completo
0,Reglamento de la Ley de Centros Penitenciarios...,True,6D0C4493_159,6D0C4493,42
1,Ley Procesal Electoral de la Ciudad de Mexico,True,3F66D749_101QUATER,3F66D749,76
2,Reglamento de la Ley de Gestion Integral de Ri...,True,D61F4C69_188,D61F4C69,185
3,Reglamento de la Ley de Gestion Integral de Ri...,True,D61F4C69_190,D61F4C69,185
4,Reglamento de la Ley de Proteccion y Fomento a...,True,69638137_12,69638137,36
...,...,...,...,...,...
5811,Reglamento de la Ley Ambiental del Distrito Fe...,True,D73158BE_86,D73158BE,85
5812,Reglamento de la Ley Ambiental del Distrito Fe...,True,D73158BE_88,D73158BE,87
5813,Reglamento de la Ley Ambiental del Distrito Fe...,True,D73158BE_93,D73158BE,92
5814,Reglamento de la Ley Ambiental del Distrito Fe...,True,D73158BE_97,D73158BE,96


In [ ]:
auto_referencia = df_llm_results[df_llm_results['es_autorreferencia'] == True].copy()
auto_referencia.head()


no_auto_referencia = df_llm_results[df_llm_results['es_autorreferencia'] == False]
no_auto_referencia.head()
df_llm_results.to_csv('../../data/05_output/art_llm_results.csv', index=False)


,ley_identificada,es_autorreferencia,art_id,doc_id,articulo_completo
43,Ley de Acceso de las Mujeres a una Vida Libre ...,False,FB6CCCAE_7,FB6CCCAE,69 Ter
47,Código Penal para el Distrito Federal,False,74FF23BF_6,74FF23BF,135 Bis
50,Código Penal del Distrito Federal,False,74FF23BF_79,74FF23BF,69 Ter
64,Ley General en Materia de Desaparición Forzada...,False,FC3F5431_253,FC3F5431,27
69,Código Penal para el Distrito Federal,False,C0E713ED_11BIS,C0E713ED,213


In [23]:
# Matchear las menciones con el LLM
import sys
sys.path.append('../functions')
from openai_law_matcher import (
    filter_official_regex_matches_parallel,
    apply_openai_law_matching_deduplicated
)

# Verificar que la API key está configurada
api_key = os.getenv('OPENAI_API_KEY')

# Inicializar cliente de OpenAI
client = OpenAI(api_key=api_key)

In [24]:
leyes_completas = pd.read_csv('../../data/02_catalogs/leyes_hash.csv')

In [25]:
# Matching deduplicado
matching_results = apply_openai_law_matching_deduplicated(
    mentions_df=no_auto_referencia,
    cdmx_laws_df=leyes_completas,
    client=client,
    entity_text_col='ley_identificada',
    art_id_col='art_id',
    delay_seconds=1.5,
    temperature=0.2
)

# Mergear con df_llm_results para conservar todas las columnas originales
results_df = matching_results.merge(
    df_llm_results,
    on='art_id',
    how='left',
    suffixes=('_match', '_original')
)

print(f"\nMerge completado:")
print(f"   • Filas en matching_results: {len(matching_results):,}")
print(f"   • Filas en df_llm_results: {len(df_llm_results):,}")
print(f"   • Filas en results_df (final): {len(results_df):,}")
print(f"   • Columnas totales: {len(results_df.columns)}")
print(f"\nColumnas finales: {results_df.columns.tolist()}")


=== MATCHING DE LEYES CON OPENAI (OPTIMIZADO - DEDUPLICADO) ===
Total de menciones (con duplicados): 661
Filas válidas: 661 (de 661 total)

Menciones ÚNICAS a procesar: 198
 Reducción: 463 llamadas ahorradas
Ahorro de tiempo estimado: 11.6 minutos
Tiempo estimado total: 5.0 minutos
Leyes oficiales CDMX en catálogo: 783
Temperatura del modelo: 0.2

Procesando menciones únicas...
  Progreso: 10/198 menciones únicas procesadas
  Tiempo transcurrido: 0.6 minutos
  Tiempo restante estimado: 4.7 minutos

  Progreso: 20/198 menciones únicas procesadas
  Tiempo transcurrido: 1.2 minutos
  Tiempo restante estimado: 4.5 minutos

  Progreso: 30/198 menciones únicas procesadas
  Tiempo transcurrido: 2.3 minutos
  Tiempo restante estimado: 4.2 minutos

  Progreso: 40/198 menciones únicas procesadas
  Tiempo transcurrido: 3.1 minutos
  Tiempo restante estimado: 4.0 minutos

  Progreso: 50/198 menciones únicas procesadas
  Tiempo transcurrido: 4.1 minutos
  Tiempo restante estimado: 3.7 minutos

  Pr

In [26]:
results_df.head()

,art_id,entity_text,cdmx_official_name,cdmx_doc_id,match_quality,openai_response,ley_identificada,es_autorreferencia,doc_id,articulo_completo
0,FB6CCCAE_7,Ley de Acceso de las Mujeres a una Vida Libre ...,Ley de Acceso de las Mujeres a una Vida Libre ...,74FF23BF,safe,MATCH: 74FF23BF,Ley de Acceso de las Mujeres a una Vida Libre ...,False,FB6CCCAE,69 Ter
1,74FF23BF_6,Código Penal para el Distrito Federal,Codigo Penal para el Distrito Federal,FC3F5431,safe,MATCH: FC3F5431,Ley de Acceso de las Mujeres a una Vida Libre ...,True,74FF23BF,200
2,74FF23BF_6,Código Penal para el Distrito Federal,Codigo Penal para el Distrito Federal,FC3F5431,safe,MATCH: FC3F5431,Ley de Acceso de las Mujeres a una Vida Libre ...,True,74FF23BF,193
3,74FF23BF_6,Código Penal para el Distrito Federal,Codigo Penal para el Distrito Federal,FC3F5431,safe,MATCH: FC3F5431,Ley de Acceso de las Mujeres a una Vida Libre ...,True,74FF23BF,152
4,74FF23BF_6,Código Penal para el Distrito Federal,Codigo Penal para el Distrito Federal,FC3F5431,safe,MATCH: FC3F5431,Código Penal para el Distrito Federal,False,74FF23BF,135 Bis


In [27]:
# Pegar el doc_id matcheado con el articulo_completo
results_df['art_id_buscar'] = results_df['cdmx_doc_id'] + '_' + results_df['articulo_completo']
results_df.head()
print(results_df.shape)



(2200, 11)


In [28]:
# Buscar si el art_id existe en el catalogo de articulos
results_df['existe_en_catalogo'] = results_df['art_id_buscar'].isin(articles_df['art_id'])
results_df.head()

# Filtrar para quedarnos solo con los que existen en el catalogo
results_df = results_df[results_df['existe_en_catalogo']]
results_df.head()

results_df['type_of_relation'] = 'mencion_articulo'
results_df = results_df[['art_id','type_of_relation','art_id_buscar']]


In [29]:
# Crear tabla final de autoreferencia
auto_referencia['art_id_buscar'] = auto_referencia['doc_id'] + '_' + auto_referencia['articulo_completo']
auto_referencia['type_of_relation'] = 'auto_referencia_articulo'
auto_referencia = auto_referencia[['art_id','type_of_relation','art_id_buscar']]


# Unir las auto referencias con los matcheados 
results_df = pd.concat([auto_referencia, results_df])
results_df['art_id_buscar'] = results_df['art_id_buscar'].str.replace(' ', '').str.upper()
results_df['art_id'] = results_df['art_id'].str.replace(' ', '').str.upper()
results_df.head()
#results_df.to_csv('../../data/04_matched/article_mentions_matched.csv', index=False, encoding='utf-8-sig')

,art_id,type_of_relation,art_id_buscar
0,6D0C4493_159,auto_referencia_articulo,6D0C4493_42
1,3F66D749_101QUATER,auto_referencia_articulo,3F66D749_76
2,D61F4C69_188,auto_referencia_articulo,D61F4C69_185
3,D61F4C69_190,auto_referencia_articulo,D61F4C69_185
4,69638137_12,auto_referencia_articulo,69638137_36


In [30]:
results_df.shape

(6628, 3)

In [31]:
# Tabla final a guardar
results_df.to_csv('../../data/04_matched/article_mentions_matched.csv', index=False, encoding='utf-8-sig')
results_df.to_csv('../../data/05_output/art_art.csv', index=False, encoding='utf-8-sig')